In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

# HIST_PRICE_DF = pd.read_excel (r'/Users/joezhou/Downloads/ALL_Prices.xlsx')
HIST_PRICE_DF_RAW = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_Prices.csv', sep='|')

#Create a sample for code testing purposes
# HIST_PRICE_DF = HIST_PRICE_DF_RAW[HIST_PRICE_DF_RAW['TickName'].isin(['BHP.AX','RIO.AX','TCL.AX','CBA.AX','MQG.AX','CSL.AX','NAB.AX','WBC.AX','SCG.AX','ANZ.AX','FMG.AX','WES.AX','TLS.AX','RMD.AX','WOW.AX','APA.AX','ATM.AX','GMG.AX','STO.AX','AMC.AX','MGR.AX','COL.AX','ALL.AX','NCM.AX','ZIP.AX'])]

HIST_PRICE_DF = HIST_PRICE_DF_RAW

#create a baselist for adding on the feature engineered variables
TICKER_LIST = HIST_PRICE_DF['TickName'].drop_duplicates().reset_index(drop=True)

In [2]:

import numpy as np

# find ends of the data window
HIST_PRICE_DF_FIRST = HIST_PRICE_DF.groupby("TickName").first()
HIST_PRICE_DF_FIRST.rename(columns={"Open": "First_Open_price"}, inplace = True)


HIST_PRICE_DF_LAST = HIST_PRICE_DF.groupby("TickName").last()
HIST_PRICE_DF_LAST.rename(columns={"Close": "Last_Close_price"}, inplace = True)
HIST_PRICE_DF = HIST_PRICE_DF.merge(HIST_PRICE_DF_LAST[['Last_Close_price']], on = 'TickName',how = 'left')


# create calculated columns
HIST_PRICE_DF['Freq'] = 1

HIST_PRICE_DF['DAY_UP'] = np.where(HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] > 0, 1, 0)
HIST_PRICE_DF['DAY_DWN'] = np.where(HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] < 0, 1, 0)
HIST_PRICE_DF['DAY_FLAT'] = np.where(HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] == 0, 1, 0)

HIST_PRICE_DF['DAY_CHANGE_RATE'] = HIST_PRICE_DF['Close'] / HIST_PRICE_DF['Open'] - 1
HIST_PRICE_DF['DAY_CHANGE_NUM'] = HIST_PRICE_DF['Close'] - HIST_PRICE_DF['Open'] 

HIST_PRICE_DF['DAYS_ABOVE_LCLSE'] = np.where(HIST_PRICE_DF['Last_Close_price'] - HIST_PRICE_DF['Close'] > 0, 1, 0)
HIST_PRICE_DF['DAYS_BELOW_LCLSE'] = np.where(HIST_PRICE_DF['Last_Close_price'] - HIST_PRICE_DF['Close'] < 0, 1, 0)

# aggregate to ticker level for calcs
HIST_PRICE_DF_SUM = HIST_PRICE_DF.groupby("TickName")["Freq", "DAY_UP","DAY_DWN", "DAY_FLAT", "DAY_CHANGE_NUM", "DAY_CHANGE_RATE", "DAYS_ABOVE_LCLSE", "DAYS_BELOW_LCLSE"].sum()



<ipython-input-2-a80d6e694314>:27: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  HIST_PRICE_DF_SUM = HIST_PRICE_DF.groupby("TickName")["Freq", "DAY_UP","DAY_DWN", "DAY_FLAT", "DAY_CHANGE_NUM", "DAY_CHANGE_RATE", "DAYS_ABOVE_LCLSE", "DAYS_BELOW_LCLSE"].sum()


In [3]:
# create ticker level file

DF_PRICE_INFO = HIST_PRICE_DF_SUM[["Freq", "DAY_UP","DAY_DWN", "DAY_FLAT", "DAYS_ABOVE_LCLSE", "DAYS_BELOW_LCLSE"]].merge(HIST_PRICE_DF_FIRST[['First_Open_price']], on = 'TickName',how = 'left')
DF_PRICE_INFO = DF_PRICE_INFO.merge(HIST_PRICE_DF_LAST[['Last_Close_price']], on = 'TickName',how = 'left')

# calculate at ticker level
DF_PRICE_INFO['PROB_HIGH'] = DF_PRICE_INFO['DAY_UP'] / DF_PRICE_INFO['Freq']
DF_PRICE_INFO['PROB_DWN'] = DF_PRICE_INFO['DAY_DWN'] / DF_PRICE_INFO['Freq']
DF_PRICE_INFO['PROB_FLAT'] = DF_PRICE_INFO['DAY_FLAT'] / DF_PRICE_INFO['Freq']

DF_PRICE_INFO['PROB_DAY_ABOVE_LCLSE'] = DF_PRICE_INFO['DAYS_ABOVE_LCLSE'] / DF_PRICE_INFO['Freq']
DF_PRICE_INFO['PROB_DAY_BELOW_LCLSE'] = DF_PRICE_INFO['DAYS_BELOW_LCLSE'] / DF_PRICE_INFO['Freq']

DF_PRICE_INFO['CHG_RTE_PRICE'] = DF_PRICE_INFO['Last_Close_price'] / DF_PRICE_INFO['First_Open_price'] -1

del HIST_PRICE_DF_SUM, HIST_PRICE_DF_FIRST, HIST_PRICE_DF_LAST

In [4]:
DF_YFIN_INFO1 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_1.csv',sep='|') 
DF_YFIN_INFO2 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_2.csv',sep='|') 
DF_YFIN_INFO3 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_3.csv',sep='|') 
DF_YFIN_INFO4 = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_INFO_4.csv',sep='|') 

DF_YFIN_INFO = pd.concat([DF_YFIN_INFO1, DF_YFIN_INFO2, DF_YFIN_INFO3, DF_YFIN_INFO4], ignore_index=True)
# DF_YFIN_INFO = pd.concat([DF_YFIN_INFO1, DF_YFIN_INFO2, DF_YFIN_INFO3], ignore_index=True)


del DF_YFIN_INFO1, DF_YFIN_INFO2, DF_YFIN_INFO3, DF_YFIN_INFO4



In [5]:
# Only keep columns needed, selected after manual review

DF_YFIN_INFO_SUBSET_VAR = DF_YFIN_INFO[[
# Price / volume Movement	
'52WeekChange','fiftyDayAverage','fiftyTwoWeekHigh','fiftyTwoWeekLow','averageDailyVolume10Day','averageVolume10days',
# Dividend	
'dividendRate','dividendYield','exDividendDate','payoutRatio','trailingAnnualDividendRate','trailingAnnualDividendYield',
# Timing	
# 'mostRecentQuarter','lastFiscalYearEnd','lastSplitDate','nextFiscalYearEnd',
# Fundamentals	
'open','marketCap','sharesOutstanding','floatShares','bookValue','regularMarketPrice','regularMarketVolume','heldPercentInsiders','heldPercentInstitutions',
# Performance	
'earningsQuarterlyGrowth','netIncomeToCommon','beta','enterpriseToEbitda','enterpriseToRevenue','enterpriseValue','priceToBook','priceToSalesTrailing12Months','profitMargins','trailingEps','trailingPE','fullTimeEmployees',
# Future dated	
'forwardEps','forwardPE',
# Company Identifier	
'symbol'
]]

DF_YFIN_INFO_SUBSET_VAR.rename(columns={"symbol": "TickName"}, inplace = True)



DF_YFIN_INFO_SUBSET_CHAR = DF_YFIN_INFO[[	
#Location	
'address1','address2','city','state','country','zip','exchange',
# Company Identifier	
'symbol','shortName','longName',
# ,'longBusinessSummary','website','messageBoardId',
# Index Grouping	
'industry','sector'
]]

DF_YFIN_INFO_SUBSET_CHAR.rename(columns={"symbol": "TickName"}, inplace = True)




#Create a sample for code testing purposes
# DF_YFIN_INFO_SUBSET_VAR = DF_YFIN_INFO_SUBSET_VAR[DF_YFIN_INFO_SUBSET_VAR['TickName'].isin(['BHP.AX','RIO.AX','TCL.AX','CBA.AX','MQG.AX','CSL.AX','NAB.AX','WBC.AX','SCG.AX','ANZ.AX','FMG.AX','WES.AX','TLS.AX','RMD.AX','WOW.AX','APA.AX','ATM.AX','GMG.AX','STO.AX','AMC.AX','MGR.AX','COL.AX','ALL.AX','NCM.AX','ZIP.AX'])]
# DF_YFIN_INFO_SUBSET_CHAR = DF_YFIN_INFO_SUBSET_CHAR[DF_YFIN_INFO_SUBSET_CHAR['TickName'].isin(['BHP.AX','RIO.AX','TCL.AX','CBA.AX','MQG.AX','CSL.AX','NAB.AX','WBC.AX','SCG.AX','ANZ.AX','FMG.AX','WES.AX','TLS.AX','RMD.AX','WOW.AX','APA.AX','ATM.AX','GMG.AX','STO.AX','AMC.AX','MGR.AX','COL.AX','ALL.AX','NCM.AX','ZIP.AX'])]


# DF_YFIN_INFO_SUBSET = DF_YFIN_INFO_SUBSET[['TickName', 'sector']]

In [6]:
# Add info onto prices data, add more features into this cell


DF_YFIN_INFO_SUBSET_VAR.set_index('TickName')
# DF_PRICE_INFO.set_index('TickName')


ADDED_PRICE_INF = DF_PRICE_INFO.merge(DF_YFIN_INFO_SUBSET_VAR, on = 'TickName',how = 'left')
# ADDED_PRICE_INF.to_excel('/Users/joezhou/Downloads/ADDED_PRICE_INF.xlsx', engine='xlsxwriter')  



In [38]:
DF_DIV_HIST = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_Div.csv',sep='|') 


# DF_DIV_HIST_S = DF_DIV_HIST[DF_DIV_HIST['TickName'].isin(['CBA.AX','WES.AX'])]

# find the last 5 year median dividend payout amount

DF_DIV_HIST['DateTime'] = pd.to_datetime(DF_DIV_HIST['Date'], format='%Y-%m-%d')
DF_DIV_HIST['Year'] = DF_DIV_HIST['DateTime'].dt.year

DF_DIV_HIST_S = DF_DIV_HIST.groupby(['TickName', "Year"])['Dividends'].sum().reset_index()
DF_DIV_HIST_S.sort_index(axis=1)

DF_DIV_HIST_S2 = DF_DIV_HIST_S.groupby("TickName").tail(5)
# calculate the median as reference point
DF_DIV_MED = DF_DIV_HIST_S2.groupby(['TickName'])['Dividends'].median().reset_index()



del DF_DIV_HIST_S, DF_DIV_HIST_S2

DF_DIV_MED.to_excel('/Users/joezhou/Downloads/DF_DIV_MED.xlsx', engine='xlsxwriter')  




# calulate flat valuation based on dividend returns
# m = Number of payments per period (e.g., m=2 for semiannually payments)
# t = Number of years to maturity
# ytm = Yield to maturity (in decimals terms)
# fv = The Bond’s Face Value
# c = Coupon rate (in decimals terms)
    
# bondPrice = ((fv*c/m*(1-(1+ytm/m)**(-m*t)))/(ytm/m)) + fv*(1+(ytm/m))**(-m*t)




In [7]:
# Calculate Ben Graham intrensic valuations and revse engineer a target growth rate based on latest closing price

# the current yield on AAA corporate bonds (CURR_AAA_Y)
CURR_AAA_Y = 3.38
# the average yield of AAA corporate bonds (AVG_BND_Y)
AVG_BND_Y = 4.4
# EPS of non growing company (EPS_NG)
EPS_NG = 8.5

# reasonably expected 7 to 10 Year Growth Rate of EPS (EPS_G = 8.5)
EPS_G = ADDED_PRICE_INF['earningsQuarterlyGrowth']*3

# actual formula
# ADDED_PRICE_INF['CALC_BF_IV'] = ADDED_PRICE_INF['trailingEps']*(EPS_NG+2*EPS_G))*AVG_BND_Y/CURR_AAA_Y
# conservative
ADDED_PRICE_INF['CALC_BF_IV'] = ADDED_PRICE_INF['trailingEps']*(EPS_NG+EPS_G)*AVG_BND_Y/CURR_AAA_Y

ADDED_PRICE_INF['CALC_BF_GRWTH'] = (ADDED_PRICE_INF['Last_Close_price']*CURR_AAA_Y/AVG_BND_Y/ADDED_PRICE_INF['trailingEps']-EPS_NG)/2



In [11]:
import pandas as pd

# FIN = pd.read_excel (r'/Users/joezhou/Downloads/ALL_FIN.xlsx') 
# FIN = pd.read_csv (r'/Users/joezhou/Downloads/R_ALL_FIN.csv',sep='|') 
FIN = pd.read_csv (r'/Users/joezhou/Downloads/ALL_FIN.csv',sep='|') 


#Create a sample for code testing purposes
FIN_S = FIN[FIN['TickName'].isin(['WES.AX'])]
FIN_S = FIN_S.loc[(FIN_S['Date'] == '2020-06-30')]

#create a baselist for adding on the feature engineered variables
# TICKER_LIST = FIN['TickName'].drop_duplicates().reset_index(drop=True)

In [22]:
# Caclulate WACC for Westfarmers

FIN_S_T = FIN_S.pivot_table('Value', ['TickName', 'Date'], 'Metric')

# FIN_S_T.to_excel('/Users/joezhou/Downloads/test_file.xlsx', engine='xlsxwriter')  

FIN_S_T['C_TAX_R'] = FIN_S_T['Income Tax Expense']/ FIN_S_T['Income Before Tax']
FIN_S_T['C_TOT_DEBT'] = FIN_S_T['Short Long Term Debt']+FIN_S_T['Long Term Debt']
FIN_S_T['C_TOT_CPTL'] = FIN_S_T['C_TOT_DEBT'] + FIN_S_T['Total Stockholder Equity']

FIN_S_T['C_TOT_DEBT_PERC'] = FIN_S_T['C_TOT_DEBT'] / FIN_S_T['C_TOT_CPTL']
FIN_S_T['C_TOT_EQTY_PERC'] = 1 - FIN_S_T['C_TOT_DEBT_PERC']

FIN_S_T['C_COST_DEBT'] = FIN_S_T['Long Term Debt'] / FIN_S_T['Total Assets']
#need to work out cost of equity using westfarmers excel sheet
FIN_S_T['C_COST_EQTY'] = 0.0502

FIN_S_T['C_WACC'] = (FIN_S_T['C_TOT_DEBT_PERC']* FIN_S_T['C_COST_DEBT']) + (FIN_S_T['C_TOT_EQTY_PERC']*FIN_S_T['C_COST_EQTY'])*(1-FIN_S_T['C_TAX_R'])


In [14]:
# extract variables and formatting
FIN_S2 = FIN_S[FIN_S['Metric'].isin(['Short Long Term Debt','Total Stockholder Equity'])]
# the above used to calculate debt and equity rate

# risk free rate, use the 10 year bond rate
RF = .01

# BETA = use from the info section

# Gross yield
average_return = 5.63




In [11]:
# Find correlating variables
df = ADDED_PRICE_INF

import numpy as np

# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.to_excel('/Users/joezhou/Downloads/corr_matrix.xlsx', engine='xlsxwriter')  

# Find features with correlation greater than 0.7
to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]

# Drop features 
df.drop(to_drop, axis=1, inplace=True)

In [62]:
# cluster to find similar performing

import matplotlib.pyplot as plt
# from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler


In [67]:
features, true_labels = make_blobs(
n_samples=200,
centers=3,
cluster_std=2.75,
random_state=42
 )

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

kmeans = KMeans(
   init="random",
   n_clusters=3,
   n_init=10,
   max_iter=300,
   random_state=42
   )


kmeans.fit(scaled_features)

KMeans(init='random', n_clusters=3, random_state=42)

In [ ]:
# Build sector related models
MOD_DATA = ADDED_PRICE_INF[['TickName','PROB_HIGH','CHG_RTE_PRICE','sector']]

# padding rows into columns of 1 or 0
# MOD_DATA = pd.concat([MOD_DATA, pd.get_dummies(MOD_DATA['sector'])], axis=1);


# transpose for excel movement correlations


MOD_DATA2 = MOD_DATA.pivot_table('PROB_HIGH', ['TickName','CHG_RTE_PRICE'], 'sector')

#Export version for Tableau

MOD_DATA2.to_excel('/Users/joezhou/Downloads/T_SECTOR_MODEL2.xls', index=False)


# del MOD_DATA['sector'], MOD_DATA['TickName']

# np.array(MOD_DATA, dtype=bool)


In [8]:
# split into dependent and independent vars
# x_columns  = MOD_DATA.iloc[: , 4:]
# y = MOD_DATA["PROB_HIGH"]


## creating function to get model statistics
# import numpy as np
# import statsmodels.api as sm
# def get_stats():
#     x = MOD_DATA[x_columns]
#     results = sm.OLS(y, x).fit()
#     print(results.summary())
# get_stats()
